In [289]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## Q2.1

In [482]:
# import the data
traindata = pd.read_csv("adult.data", header=None, sep=', ')
testdata = pd.read_csv("adult.test.txt", header = None,  sep=', ')

# add the column name
testdata.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

traindata.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

In [483]:
# Let's convert the <=50Ks into 0 and the >50K into +1
traindata["Income"] = traindata["Income"].map({ "<=50K": 0, ">50K": 1 })
testdata["Income"] = testdata["Income"].map({ "<=50K.": 0, ">50K.": 1 })

In [484]:
# remove the missing values
# train: 30162 row data; test 15060 row data
for i in traindata.columns:
    traindata = traindata[traindata[i] != '?'] 
    testdata = testdata[testdata[i] != '?'] 

In [485]:
# "1-of-K" encoding
traindata = pd.get_dummies(traindata, columns=[
    "WorkClass", "Education", "MaritalStatus", "Occupation", "Relationship","Race", "Gender", "NativeCountry",])
testdata = pd.get_dummies(testdata, columns=[
    "WorkClass", "Education", "MaritalStatus", "Occupation", "Relationship","Race", "Gender", "NativeCountry",])

In [486]:
# drop the features which frequency < 11 of trainning data
for i in traindata.columns:
    total = traindata[i].sum()
    if total < 11:
        traindata = traindata.drop([i], axis=1)

# drop the columns which are not in the traindata
for i in testdata.columns:
    if i not in traindata.columns:
        testdata = testdata.drop([i], axis=1)

In [487]:
# check the shape of dataset
print("train:", traindata.shape, "test:", testdata.shape)

train: (30162, 103) test: (15060, 103)


In [488]:
# re-organize the data
x_train = traindata.drop(['Income'], axis=1)
y_train = traindata['Income'] 
x_test = testdata.drop(['Income'], axis=1)
y_test = testdata['Income'] 

(a) Summary statistics of train data:

In [270]:
print(x_train.describe())
print("\nLabel counts:")
print(y_train.value_counts())

                Age        fnlwgt  EducationNum   CapitalGain   CapitalLoss  \
count  30162.000000  3.016200e+04  30162.000000  30162.000000  30162.000000   
mean      38.437902  1.897938e+05     10.121312   1092.007858     88.372489   
std       13.134665  1.056530e+05      2.549995   7406.346497    404.298370   
min       17.000000  1.376900e+04      1.000000      0.000000      0.000000   
25%       28.000000  1.176272e+05      9.000000      0.000000      0.000000   
50%       37.000000  1.784250e+05     10.000000      0.000000      0.000000   
75%       47.000000  2.376285e+05     13.000000      0.000000      0.000000   
max       90.000000  1.484705e+06     16.000000  99999.000000   4356.000000   

       HoursPerWeek  WorkClass_Federal-gov  WorkClass_Local-gov  \
count  30162.000000           30162.000000         30162.000000   
mean      40.931238               0.031265             0.068530   
std       11.979984               0.174035             0.252657   
min        1.000000 

(b) Summary statistics of test data:

In [272]:
print(x_test.describe())
print("\nLabel counts:")
print(y_test.value_counts())

                Age        fnlwgt  EducationNum   CapitalGain   CapitalLoss  \
count  15060.000000  1.506000e+04  15060.000000  15060.000000  15060.000000   
mean      38.768327  1.896164e+05     10.112749   1120.301594     89.041899   
std       13.380676  1.056150e+05      2.558727   7703.181842    406.283245   
min       17.000000  1.349200e+04      1.000000      0.000000      0.000000   
25%       28.000000  1.166550e+05      9.000000      0.000000      0.000000   
50%       37.000000  1.779550e+05     10.000000      0.000000      0.000000   
75%       48.000000  2.385888e+05     13.000000      0.000000      0.000000   
max       90.000000  1.490400e+06     16.000000  99999.000000   3770.000000   

       HoursPerWeek  WorkClass_Federal-gov  WorkClass_Local-gov  \
count  15060.000000           15060.000000         15060.000000   
mean      40.951594               0.030744             0.068592   
std       12.062831               0.172628             0.252768   
min        1.000000 

# Q2.2

以本題求的error function為例，可以導出gradient 與hessian 如下：
1. gradient() <br>
lambda_matrix: 對角矩陣的lambda值 <br>
w: coefficient of features <br>
x_train: trainning data of features <br>
y_train: trainning data of labels <br>
p: sigmoid 轉換後得出之機率 <br>
<br>
2. hessian() <br>
lambda_matrix: 對角矩陣的lambda值 <br>
w: coefficient of features <br>
x_train: trainning data of features <br>
p: sigmoid 轉換後得出之機率 <br>

In [280]:
def gradient(lambda_matrix, w, x_train, y_train, p):
    g = lambda_matrix @ w + x_train @ (p - y_train)   
    
    return g

def hessian(lambda_matrix, w, x_train, p):
    H = lambda_matrix @ np.identity(lambda_matrix.shape[0]) + x_train.T @ np.diag(mat(p*(1-p)).getA1()) @ x_train  
    
    return H

# Q2.3

In [489]:
class mylogistic_l2():
    
    def __init__(self, reg_vec, max_iter, tol, add_intercept):
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept
        return
        

    def sigmoid(self, x):
        p_matrix = 1.0/(1.0+exp(-x))
        p_array = np.squeeze(p_matrix)
        p_array[p_array > 0.99] = 0.999999
        p_array[p_array <= 0] = 0.000001

        p_matrix2 = p_array.reshape(p_matrix.shape[0], p_matrix.shape[1])

        return p_matrix2

    def fit(self, x_train , y_train):
        self.x_train = x_train.copy()
        self.y_train = y_train.copy()
        
        samples, features = self.x_train.shape
        
        
        # check if need intercept then features would puls 1
        if self.add_intercept == True:
            self.x_train = np.hstack((self.x_train, np.ones((self.x_train.shape[0],1))))

            
        self.y_train = self.y_train.reshape(samples,1)

        # create lambda diagnoal marix
        lambda_matrix = np.diag(self.reg_vec)


        # use closed form sol of ridge regression as initail value of coefficient vector: w
        b = np.mean(self.reg_vec)
        self.w = np.linalg.inv(self.x_train.T @ self.x_train + b * np.identity(1)) @ self.x_train.T @ self.y_train

        # other initial values
        iternum = 0 
        loss0 = float('inf')
        
        while iternum < self.max_iter:
                p = self.sigmoid(self.x_train @ self.w) # MLE
                
                g = lambda_matrix @ self.w + self.x_train.T @ (p - self.y_train)   # gradient
                # hessian
                H = lambda_matrix @ np.identity(lambda_matrix.shape[0]) + self.x_train.T @ np.diag(mat(p*(1-p)).getA1()) @ self.x_train  

                loss = (self.w.T @ lambda_matrix @ self.w) - sum(self.y_train.T @ np.log(p) + (1 - self.y_train).T @ np.log(1-p)) 

                self.w = self.w -  np.linalg.inv(H) @ g  # update w
                iternum += 1
               
                
                if loss0 - loss < self.tol:
                    break
                    
                loss0 = loss
                
                print(iternum, loss0, loss, loss0-loss)
                                
        return self.w


    def predict(self, x_test):
        
        if self.add_intercept == True:
            x_test = np.hstack((x_test, np.ones((x_test.shape[0],1)))) 
            
        y_pred = []
        y_array = np.squeeze(self.sigmoid(x_test @ self.w))

        for i in range(len(y_array)):
            if i >= 0.5:
                tmp = 1
                y_pred.append(tmp)
            else:
                tmp = 0
                y_pred.append(tmp)
        
        return y_pred
        


In [491]:
# convert dataset to array type
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

We have 102 features, including: <br>
5 continuous variables <br>
97 binary variables <br>

if intercept is true <br>
then we have another feature as last column

### case1: lambda = 1 for all coefficients

In [492]:
# run the model
lambda_vec = [1 for i in range(103)]
reg = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol =0.00001, add_intercept = True)
w = reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

1 [[2.05921178e+31]] [[2.05921178e+31]] [[0.]]
2 [[843796.78073062]] [[843796.78073062]] [[0.]]


In [495]:
#Accuracy
hit = 0.
for i in range(len(y_pred)):
    if y_pred[i]== y_test[i]:
        hit += 1
        
print("Accuracy:", hit/x_test.shape[0])
print("Learned w: \n", w) # the last item is intercept

Accuracy: 0.24575033200531207
Learned w: 
 [[-1.11521617e+03]
 [ 9.25781811e-02]
 [ 1.19705998e+03]
 [ 4.48993005e-01]
 [ 1.60385820e+00]
 [ 2.26250477e+02]
 [ 8.98976911e+02]
 [-1.46984449e+03]
 [ 3.02227114e+03]
 [-6.87619374e+02]
 [-1.63733094e+03]
 [-1.27715808e+03]
 [-9.64858917e+00]
 [-5.31474510e+02]
 [-4.01641983e+02]
 [-2.77684192e+02]
 [-1.03293798e+02]
 [-2.07288276e+02]
 [-9.43697251e+02]
 [-4.21844301e+02]
 [ 3.37215987e+02]
 [ 3.26704062e+03]
 [ 2.41273650e+03]
 [-1.27281699e+01]
 [ 7.27826896e+02]
 [-2.89221460e+03]
 [-3.02250327e+01]
 [-4.26049174e+01]
 [-2.04047639e+03]
 [ 3.02585448e+02]
 [-1.05976321e+00]
 [ 6.10058972e+02]
 [-2.63398254e+02]
 [-3.38692566e+03]
 [ 2.28089217e+03]
 [-7.02506336e+02]
 [ 1.21373472e+03]
 [ 2.24104756e+02]
 [-8.79423962e+02]
 [-1.00489409e+03]
 [-9.32748398e+02]
 [-3.52298882e+02]
 [ 4.75914310e+01]
 [-9.77109933e+01]
 [ 1.33695861e+03]
 [-2.92748877e+02]
 [-7.51235356e+01]
 [ 7.16113533e+02]
 [-1.06285470e+03]
 [ 1.99664435e+03]
 [ 3.26

#### Case 2: lambda = 1 for all but the intercept, no regularization for incercept term.

In [499]:
# run the model
lambda_vec = [1 for i in range(102)] + [0]
reg = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol =0.00001, add_intercept = True)
w = reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

1 [[1.719186e+30]] [[1.719186e+30]] [[0.]]
2 [[3374356.542807]] [[3374356.542807]] [[0.]]


In [500]:
#Accuracy
hit = 0.
for i in range(len(y_pred)):
    if y_pred[i]== y_test[i]:
        hit += 1
        
print("Accuracy:", hit/x_test.shape[0])
print("Learned w: \n", w) # the last item is intercept

Accuracy: 0.24575033200531207
Learned w: 
 [[ 2.77606250e+04]
 [ 4.90404421e-01]
 [-1.57204563e+03]
 [ 5.69618969e+00]
 [ 2.25983130e+01]
 [ 5.94029420e+03]
 [-5.56430942e+01]
 [-4.06705084e+02]
 [ 1.19563692e+03]
 [-3.57548749e+01]
 [-4.81780369e+02]
 [-2.11851585e+02]
 [-3.90191212e+00]
 [-6.68553722e+01]
 [-7.12399544e+01]
 [-3.87501783e+01]
 [-2.74842374e+00]
 [-9.60389219e+00]
 [-6.97440757e+01]
 [-3.30561046e+01]
 [-2.26327865e+02]
 [-2.15207288e+02]
 [-5.65982887e+02]
 [-2.23282815e+01]
 [-5.19500071e+02]
 [-1.94799931e+02]
 [ 1.74793654e+00]
 [-5.68690065e+00]
 [ 2.04008329e+03]
 [-1.26844352e+03]
 [ 2.82246659e+00]
 [ 9.55604373e+02]
 [ 2.27793877e+03]
 [-1.43843697e+03]
 [-2.75450382e+02]
 [-2.54034734e+02]
 [-6.98868272e+02]
 [-4.77934064e+02]
 [-1.52430627e+02]
 [-2.38217231e+02]
 [-2.28402085e+02]
 [-3.01585410e+02]
 [-5.61899108e+02]
 [-2.75830982e+01]
 [-4.16092860e+02]
 [-7.64603719e+01]
 [ 3.47444912e+03]
 [-7.22710764e+01]
 [-2.20009051e+02]
 [ 8.29245199e+02]
 [-1.55

### Case 3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for incercept term.

In [504]:
# run the model
lambda_vec = [1 for i in range(5)] + [0.5 for i in range(97)] + [0]
reg = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol =0.00001, add_intercept = True)
w = reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

1 [[6.82245248e+31]] [[6.82245248e+31]] [[0.]]
2 [[2.37264683e+08]] [[2.37264683e+08]] [[0.]]


In [508]:
#Accuracy
hit = 0.
for i in range(len(y_pred)):
    if y_pred[i]== y_test[i]:
        hit += 1
        
print("Accuracy:", hit/x_test.shape[0])
print("Learned w: \n", w) # the last item is intercept

Accuracy: 0.24575033200531207
Learned w: 
 [[-3.54992966e+02]
 [ 3.33441803e-01]
 [ 7.51519586e+03]
 [ 1.78865254e+01]
 [ 1.49939006e+02]
 [ 3.70344233e+03]
 [ 1.98140025e+02]
 [ 1.05897283e+02]
 [-4.90314510e+02]
 [ 3.88672572e+02]
 [-1.68301017e+02]
 [-2.56978253e+01]
 [-8.39652847e+00]
 [-3.84600272e+02]
 [-5.08120349e+02]
 [ 2.03244241e+03]
 [-1.09353673e+02]
 [-1.71373950e+02]
 [-3.49022622e+02]
 [-2.11660055e+02]
 [-2.85289616e+01]
 [ 8.96214058e+01]
 [ 9.42851928e+02]
 [ 2.60656703e+02]
 [-1.49776992e+03]
 [ 7.86125550e+02]
 [-3.40310026e+01]
 [ 2.86404143e+02]
 [-1.10364133e+03]
 [-1.11424729e+03]
 [ 7.96122189e+00]
 [ 4.51380529e+03]
 [-1.21345530e+02]
 [-2.52680727e+03]
 [-2.92184693e+02]
 [-4.67181720e+02]
 [-9.39939051e+02]
 [-1.88330423e+02]
 [ 1.31603935e+03]
 [-3.96435125e+02]
 [-5.20579210e+02]
 [-4.01622873e+02]
 [ 6.84826132e+02]
 [-9.12288601e+01]
 [ 9.86003334e+02]
 [ 8.73578821e+01]
 [-3.39120331e+02]
 [ 5.54166159e+01]
 [-2.50840791e+02]
 [ 3.87052121e+03]
 [-2.59

## Q.2.4

In [ ]:
i = 0.01
step = 10
a1 = 0.01
a2 = 0.01

while a1 < 100:
    while a2 < 100:
        # run the model
lambda_vec = [1 for i in range(5)] + [0.5 for i in range(97)] + [0]
reg = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol =0.00001, add_intercept = True)
w = reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)
        
    
    
    
    
    
    